In [2]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader, Dataset, random_split

# Summary-like from Tensorflow
from torchsummary import summary

# Import Numpy
import numpy as np
# Plot Import
import matplotlib.pyplot as plt
# For timer
import time
# Import OS
import os
# import local libs
import ecg_tools_lite as et

from statistics import mean

# WFDB
import wfdb

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')

In [3]:
device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu')

print("Device:", device, time.localtime().tm_mon, "/", time.localtime().tm_mday, "/", time.localtime().tm_year)
device

Device: cuda 11 / 29 / 2020


device(type='cuda')

In [37]:
ecg_whole = et.load_signal('118e06', '', v_fields=True, channel=1) # Creates an array of the entire thing
# Send data to CUDA from CPU
ecg_whole = torch.from_numpy(ecg_whole).double().to(device)

# Create train and validation datasets
train_size = int( (ecg_whole.shape[0])*.80 ) # Get 80%
val_size = int( (ecg_whole.shape[0])*.20 ) # Get 20%
train_dataset, val_dataset = random_split( ecg_whole, (train_size, val_size) ) # (dataset, (80%, 20% of data ))

train_loader = DataLoader( train_dataset, shuffle=True, batch_size=360)
val_loader = DataLoader( val_dataset, shuffle=True, batch_size=360)
# ecg_loader = DataLoader( ecg_whole, batch_size=360, shuffle=True)


Printing fields information of the signal: 
{'fs': 360, 'sig_len': 650000, 'n_sig': 1, 'base_date': None, 'base_time': None, 'units': ['mV'], 'sig_name': ['V1'], 'comments': ["Created by `nst' from records 118 and em (SNR = 6 dB)"]}


In [89]:
# =============== LOAD ECG DATA ===============
# Load numpy version datasets
ecg_clean = et.load_ecg_file('all_chunks_fs1024_e24.npy')
ecg_noisy = et.load_ecg_file('all_chunks_fs1024_e06.npy')
# Load datasets to CUDA
ecg_noisy = torch.from_numpy(ecg_noisy).double().to(device)
ecg_clean = torch.from_numpy(ecg_clean).double().to(device)

# Create train and validation datasets
train_size = int( (ecg_noisy.shape[0])*.80 ) # Get 80%
val_size = int( (ecg_noisy.shape[0])*.20 ) # Get 20%

# Use this for checking
total_size = train_size + val_size
if total_size < ecg_noisy.shape[0]:
    train_size = train_size + (ecg_noisy.shape[0] - total_size)

# Split datasets into train and validation
train_dataset, val_dataset = random_split( ecg_noisy, (train_size, val_size) )

# Create a data loader for both datasets
train_loader = DataLoader( train_dataset, shuffle=True)
train_loader = DataLoader( val_dataset, shuffle=True)
# Preview the shape of the train_loader
print( f"When train_loader is called, the entire shape is: {len(train_loader)}, 1024, 1" )
print( f"When val_loader is called, the entire shape is: {len(val_loader)}, 1024, 1" )
print( f"Total size: {total_size} = {train_size} + {val_size}" )

4674
When train_loader is called, the entire shape is: 1168, 1024, 1
When val_loader is called, the entire shape is: 362, 1024, 1
Total size: 5841 = 4674 + 1168


In [84]:
for i, data in enumerate( val_loader ):
    print( i )
    # if i == 0:
    #     print(f'Shape: {data.shape}')
    #     x = data.cpu().numpy()
    #     plt.figure( figsize=(20,5) )
    #     plt.plot( x[0][:] )
    #     break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27